In [1]:
#Fill these file paths with the locations on your machine.
PATH_TO_PASTIS = 'D://content//PASTIS//PASTIS'



import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib
import os
matplotlib.use('agg')

# 构建colormap用于画图
cm = matplotlib.cm.get_cmap('tab20')
def_colors = cm.colors
cus_colors = ['k'] + [def_colors[i] for i in range(1,20)]+['w']   # 合并数组
cmap = ListedColormap(colors = cus_colors, name='agri',N=21)



def get_rgb(x):
    """Utility function to get a displayable rgb image 
    from a Sentinel-2 time series.
    """
    batch_index = 0
    im_list = []
    for t_show in range(x.shape[1]):
        im = x[batch_index, t_show, [2,1,0]].cpu().numpy() # to (128,128,10)
        mx = im.max(axis=(1,2))
        mi = im.min(axis=(1,2))   
        im = (im - mi[:,None,None])/(mx - mi)[:,None,None] # 归一化
        im = im.swapaxes(0,2).swapaxes(0,1)
        im = np.clip(im, a_max=1, a_min=0) # 将值限定在a_min到a_max之间
        im_list.append(im)
    return im_list

def save_rgb(x,dates,patch_id):
    """
    save rgb image
    """
    batch_index = 0
    for t_show in range(x.shape[1]):
        im = x[batch_index, t_show, [2,1,0]].cpu().numpy() # to (128,128,10)
        mx = im.max(axis=(1,2))
        mi = im.min(axis=(1,2))   
        im = (im - mi[:,None,None])/(mx - mi)[:,None,None] # 归一化
        im = im.swapaxes(0,2).swapaxes(0,1)
        im = np.clip(im, a_max=1, a_min=0) # 将值限定在a_min到a_max之间
        if not os.path.exists("pic/{}".format(patch_id)):
            os.mkdir("pic/{}".format(patch_id))
        plt.imsave("pic/{}/index{}date{}.jpg".format(patch_id,t_show,dates[batch_index][t_show]),im)
        

# 将时间差转换为具体日期

In [2]:
from datetime import datetime,timedelta
def dateconvert(date):
    reference_date = "2018-09-01"
    reference_date = datetime(*map(int, reference_date.split("-")))
    new_date = reference_date + timedelta(days=date)
    return new_date.strftime("%Y%m%d")

In [3]:
import sys
from dataloader import PASTIS_Dataset
sys.path.append(r"D://content//PASTIS_github")
import collate

folds = [1]

dt = PASTIS_Dataset(PATH_TO_PASTIS, norm=True, target='semantic',folds=folds)
# If you only need to evaluate semantic segmentation use target='semantic'


# Batch_size设置为1
# 用于显示每个patch的图像
dl = torch.utils.data.DataLoader(dt, batch_size=1, collate_fn=collate.pad_collate, shuffle=False)
print(dl.__len__())

Reading patch metadata . . .
Done.
Dataset ready.
487


# 显示筛选之后的图片

In [5]:
for id, batch in enumerate(dl):

    (x, dates), y, id_patch = batch
    # if item.item() > 1818:
    if True:
        # print("item{},id_patch{}".format(item[0],id_patch[0]))
        fix, axes = plt.subplots(2,2, figsize=(20,20))
        # 对于 target = semantic y 只有一个label信息
        pixel_semantic_annotation = y
        print("id_patch{}".format(id_patch[0]))
        # # x is a dictionary thay contains the Sentinel-2 time series
        # # in the 'S2' key, shape: B x T x C x H x W
        # print(x.shape)
        # # dates contains the observation dates of each image, shape: B x T
        # print(dates.shape)
        # # y contains the different ground truth annotations used for panoptic segmentation
        # # (
        # #     target_heatmap,
        # #     instance_ids,
        # #     pixel_to_instance_mapping,
        # #     instance_bbox_size,
        # #     object_semantic_annotation,
        # #     pixel_semantic_annotation,
        # # ) = y.split((1, 1, 1, 2, 1, 1), dim=-1)
        # print(pixel_semantic_annotation[0])
        # print(dates[0])

            
        im = get_rgb(x)

        for i in range(2):
            for j in range(2):
                axes[i][j].imshow(im[i * 2 + j])
                axes[i][j].set_title("date{}".format(dateconvert(dates[0][i * 2 + j].item())))

        plt.savefig("D://content//PASTIS//data//vis//Fold{}//patch{}.png".format(folds[0],id_patch.item()))
        # plt.show()
        # print(im1.shape)  #(128,128,3)
        # print(pixel_semantic_annotation[0].squeeze().size())   #(128,128)
        plt.clf()
        plt.close()
    break

id_patchtensor([10000])


# 将选择到的图像的日期以及id存入data.json
## 使用dataloader_date.py文件

In [24]:
import json
load_dict = {}
features = []
for id, batch in enumerate(dl):

    (x, dates), y, item, id_patch, date = batch
    # print(type(dateconvert(dates[0][0].item())))
    patch_dict = {}
    date_list = date[0].tolist()
    patch_dict["ID_PATCH"] = "{}".format(id_patch.item())
    date_dict = {}
    for i,date_id in enumerate(date_list):
        date_dict["{}".format(date_id)] = dateconvert(dates[0][i].item())
    patch_dict["DATE"] = date_dict
    features.append(patch_dict)
load_dict["features"] = features
with open("data.json", 'w') as write_f:
	json.dump(load_dict, write_f, indent=4, ensure_ascii=False)

In [28]:
a = {"1":1,"2":2}

['1', '2']